In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from transformers import AutoModel
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


In [3]:
df_allCarabids = pd.read_csv("/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv", sep="\t")
df_insect1M = df_allCarabids[df_allCarabids["filepath"].str.startswith("Insect-1M")].copy()
df_insect1M.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = df_insect1M[cols]
df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']
df = df.fillna("Unknown")
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model(image_tensor).last_hidden_state.mean(dim=1)

    
    return features.cpu().numpy()
    

In [6]:
# Separate known and unknown samples
df_known = df[df["ScientificName"] != "Unknown"]
df_unknown = df[df["ScientificName"] == "Unknown"]

# Extract features for known samples (only passing img)
X_known = np.vstack([extract_features(img) for img in tqdm(df_known["ImageFilePath"], desc="Processing ...")])

# Encode labels only for known species
le = LabelEncoder()
y_known = le.fit_transform(df_known["ScientificName"])

df_indices_known = df_known.index

# Train-test split only on known species
X_train, X_test_known, y_train, y_test_known, train_idx, test_idx_known = train_test_split(
    X_known, y_known, df_indices_known, test_size=0.2, random_state=42
)

# Extract features for unknown samples (only passing img)
X_test_unknown = np.vstack([extract_features(img) for img in tqdm(df_unknown["ImageFilePath"], desc="Processing Unknown ...")])
test_idx_unknown = df_unknown.index

# Assign a placeholder label (-1) for unknown samples
y_test_unknown = np.full(len(X_test_unknown), -1)

# Combine known and unknown test sets
X_test = np.vstack([X_test_known, X_test_unknown])
y_test = np.concatenate([y_test_known, y_test_unknown])
test_idx = np.concatenate([test_idx_known, test_idx_unknown])

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")


Processing Unknown ...: 100%|██████████| 4328/4328 [01:13<00:00, 59.28it/s]

Train set: (16222, 768), Test set: (8384, 768)


In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [9]:


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():

    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 5.47% | Prec: 5.03% | Rec: 5.47% | F1: 3.95% | Bal Acc: 8.47% | MCC: 0.0630
LogisticRegression        | Acc: 21.41% | Prec: 13.65% | Rec: 21.41% | F1: 15.16% | Bal Acc: 27.82% | MCC: 0.2489
NearestNeighbor           | Acc: 8.03% | Prec: 6.77% | Rec: 8.03% | F1: 6.36% | Bal Acc: 9.50% | MCC: 0.0926
MLP_Baseline              | Acc: 18.74% | Prec: 12.60% | Rec: 18.74% | F1: 13.83% | Bal Acc: 23.70% | MCC: 0.2177


In [11]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
8222,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela willistoni,Ellipsoptera nevadica,Cicindela willistoni,Cicindela amargosae,Cicindelidia haemorrhagica
15309,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis


In [12]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M/7.DINOv2-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M/7.DINOv2-linear-probing-species-metrics.csv", index=False)